In [19]:
# Importing neccasary Libraries 
import pandas as pd
import numpy as np 
%pip install upgini 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


PREPARE THE DATASET

In [20]:
from os.path import exists
df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df = df.sample(n=19_000, random_state=0)

In [21]:
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)
df["date"] = pd.to_datetime(df["date"])

df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop=True)


SPLIT THE DATASET TO TRAIN AND TEST DATASET

In [22]:
# Putting data from 2013 till 2016 in the training dataset then 2017 data in the test data set 

train = df[df['date'] < '2017-1-1']
test = df[df['date'] > '2016-12-31']

#Splitting test and train dataset into feautures and labels 

train_features = train.drop(columns = ['sales'], axis = 1)
train_target = train['sales']

test_features = test.drop(columns = ['sales'], axis = 1)
test_target = test['sales']

ENRICH THE DATASET USING UPGINI

In [23]:
from upgini import  FeaturesEnricher, SearchKey 
from upgini.metadata import CVType

enricher = FeaturesEnricher(
        search_keys = {
        'date' : SearchKey.DATE,
        },
    cv = CVType.time_series 
)

enricher.fit(train_features,
            train_target, 
            eval_set = [(test_features, test_target)])

Detected task type: ModelTaskType.REGRESSION


Column name,Status,Description
date,All valid,All values in this column are good to go
target,All valid,All values in this column are good to go


Running search request with search_id=3a1a2a9f-4cca-4c96-825c-ef3e1657d8c7
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done


28 relevant feature(s) found with the search keys: ['date'].


,feature_name,shap_value,coverage %,type
0,item,0.487726,100.0,CHARACTER
1,store,0.172106,100.0,CHARACTER
2,f_weather_pca_0_94efd18d,0.056047,100.0,NUMERIC
3,f_week_sin1_a71d22f6,0.044632,100.0,NUMERIC
4,f_week_cos1_d3d56d7f,0.029552,100.0,NUMERIC
5,f_weather_umap_48_66a91289,0.025132,100.0,NUMERIC
6,f_weather_umap_24_409427e4,0.019315,100.0,NUMERIC
7,f_weather_umap_33_b9760f68,0.014638,100.0,NUMERIC
8,f_year_cos1_cd165f8c,0.012112,100.0,NUMERIC
9,f_dow_jones_89547e1d,0.007461,100.0,NUMERIC


In [24]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose = False, allow_writing_files = False, random_state = 0)

enricher.calculate_metrics(
    train_features, train_target, 
    eval_set = [( test_features, test_target)],
    estimator = model,
    scoring = 'mean_absolute_percentage_error'
)

Calculating metrics...


/Users/aishwaryanagaraj/Library/Python/3.9/lib/python/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


/Users/aishwaryanagaraj/Library/Python/3.9/lib/python/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


/Users/aishwaryanagaraj/Library/Python/3.9/lib/python/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


/Users/aishwaryanagaraj/Library/Python/3.9/lib/python/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


...

/Users/aishwaryanagaraj/Library/Python/3.9/lib/python/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


/Users/aishwaryanagaraj/Library/Python/3.9/lib/python/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


/Users/aishwaryanagaraj/Library/Python/3.9/lib/python/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Users/aishwaryanagaraj/Library/Python/3.9/lib/python/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


Done


,match_rate,baseline mean_absolute_percentage_error,enriched mean_absolute_percentage_error,uplift
,,,,
train,100.0,0.255844,0.16662,0.089224
eval 1,100.0,0.243877,0.13113,0.112746


In [25]:
# Adding the enriched Dataset to the normal Dataset 

enriched_train_features = enricher.transform(train_features, keep_input = True)
enriched_test_features = enricher.transform(test_features, keep_input = True)

90.39637% of the rows are fully duplicated


Column name,Status,Description
date,All valid,All values in this column are good to go


Running search request with search_id=567def35-0448-45a9-9fe5-782a4fe167d2
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done

Done
90.36176% of the rows are fully duplicated


Column name,Status,Description
date,All valid,All values in this column are good to go


Running search request with search_id=9f688e24-093a-4aad-9ea4-db2320214922
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done

Done


In [26]:
# Training models on the normal dataset to find its accuracy using the SMAPE metric
# Symmetric mean absolute percentage error ( SMAPE )

model.fit(train_features, train_target)
preds = model.predict(test_features)
eval_metric(test_target.values, preds, 'SMAPE')

/Users/aishwaryanagaraj/Library/Python/3.9/lib/python/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[37.65141857448004]

In [27]:
# Training our enriched Data into the model to find its accuracy using the SMAPE metric  
model.fit(enriched_train_features,train_target)
enriched_preds = model.predict(enriched_test_features)

eval_metric(test_target.values, enriched_preds, 'SMAPE')

[14.504497540797917]

A significant Decrese in the error metric from 37.6% to 14.7% 